# HSV picker
이미지를 불러와서 HSV 이미지의 픽셀을 pick하면 마스킹된 이미지와 함께 lower와 upper값을 출력합니다

    Blue
    low = (96, 100,  50)
    high = (116, 255, 255)

In [35]:
import cv2
import numpy as np
import sys
import tkinter as tk
from tkinter import filedialog

image_hsv = None
pixel = (0,0,0) #RANDOM DEFAULT VALUE

ftypes = [
    ("JPG", "*.jpg;*.JPG;*.JPEG"), 
    ("PNG", "*.png;*.PNG"),
    ("GIF", "*.gif;*.GIF"),
    ("All files", "*.*")
]

def check_boundaries(value, tolerance, ranges, upper_or_lower):
    
    # tolerance
    #tolerance = 70
    
    
    if ranges == 0:
        # set the boundary for hue
        boundary = 180
    elif ranges == 1:
        # set the boundary for saturation and value
        boundary = 255

    if(value + tolerance > boundary):
        value = boundary
    elif (value - tolerance < 0):
        value = 0
    else:
        if upper_or_lower == 1:
            value = value + tolerance
        else:
            value = value - tolerance
    return value

def pick_color(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        pixel = image_hsv[y,x]

        #HUE, SATURATION, AND VALUE (BRIGHTNESS) RANGES. TOLERANCE COULD BE ADJUSTED.
        # Set range = 0 for hue and range = 1 for saturation and brightness
        # set upper_or_lower = 1 for upper and upper_or_lower = 0 for lower
        hue_upper = check_boundaries(pixel[0], 10, 0, 1)
        hue_lower = check_boundaries(pixel[0], 10, 0, 0)
        saturation_upper = check_boundaries(pixel[1], 10, 1, 1)
        saturation_lower = check_boundaries(pixel[1], 10, 1, 0)
        value_upper = check_boundaries(pixel[2], 255, 1, 1)
        value_lower = check_boundaries(pixel[2], 0, 1, 0)

        upper =  np.array([hue_upper, saturation_upper, value_upper])
        lower =  np.array([hue_lower, saturation_lower, value_lower])
        print(lower, upper)

        #A MONOCHROME MASK FOR GETTING A BETTER VISION OVER THE COLORS 
        image_mask = cv2.inRange(image_hsv,lower,upper)
        cv2.imshow("Mask",image_mask)

def main():

    global image_hsv, pixel

    #OPEN DIALOG FOR READING THE IMAGE FILE
    root = tk.Tk()
    root.withdraw() #HIDE THE TKINTER GUI
    file_path = filedialog.askopenfilename(filetypes = ftypes)
    root.update()
    image_src = cv2.imread(file_path)
    cv2.imshow("BGR",image_src)

    #CREATE THE HSV FROM THE BGR IMAGE
    image_hsv = cv2.cvtColor(image_src,cv2.COLOR_BGR2HSV)
    cv2.imshow("HSV",image_hsv)

    #CALLBACK FUNCTION
    cv2.setMouseCallback("HSV", pick_color)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__=='__main__':
    main()

[ 96 172  80] [116 192 255]
[ 96 172  77] [116 192 255]
[ 96 172  77] [116 192 255]
[140  28  47] [160  48 255]
[ 99 159  68] [119 179 255]
[ 97 155  85] [117 175 255]
[ 96 167  72] [116 187 255]
[ 93 149  61] [113 169 255]
[ 95 156  66] [115 176 255]
[140  28  47] [160  48 255]
[ 96 172  77] [116 192 255]
[ 95 173  74] [115 193 255]
[ 96 170  71] [116 190 255]
[ 96 170  71] [116 190 255]


# Vedeo Recording

In [86]:
import cv2 as cv
import numpy as np
import os


def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30),
                                     flags=cv.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects


def removeFaceAra(img, cascade):
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     gray = cv.equalizeHist(gray)
#     rects = detect(gray, cascade)

#     height,width = img.shape[:2]

#     for x1, y1, x2, y2 in rects:
#         cv.rectangle(img, (x1-10, 0), (x2+10, height), (0,0,0), -1)
    img = img

    return img


def make_mask_image(img_bgr,high,low):
    
    img_hsv = cv.cvtColor(img_bgr, cv.COLOR_BGR2HSV)

    #img_h,img_s,img_v = cv.split(img_hsv)

    low = low
    high = high
    

    img_mask = cv.inRange(img_hsv, low, high)
    
    kernel = np.ones((3,3),np.uint8)
    
    #closing = cv.morphologyEx(img_mask, cv.MORPH_CLOSE, kernel)
    #opening = cv.morphologyEx(closing, cv.MORPH_OPEN, kernel)
    
    return img_mask

  
def distanceBetweenTwoPoints(start, end):

    x1,y1 = start
    x2,y2 = end
 
    return int(np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2)))


def calculateAngle(A, B):

    A_norm = np.linalg.norm(A)
    B_norm = np.linalg.norm(B)
    C = np.dot(A,B)

    angle = np.arccos(C/(A_norm*B_norm))*180/np.pi
    return angle


def findMaxArea(contours):
  
    max_contour = None
    max_area = -1


    for contour in contours:
        area = cv.contourArea(contour)

        x,y,w,h = cv.boundingRect(contour)

        if (w*h)*0.4 > area:
            continue

        if w > h:
            continue

        if area > max_area:
            max_area = area
            max_contour = contour
  
    if max_area < 10000:
        max_area = -1

    return max_area, max_contour


# def getFingerPosition(max_contour, img_result, debug):
#     points1 = []


#   # STEP 6-1
#     M = cv.moments(max_contour)

#     cx = int(M['m10']/M['m00'])
#     cy = int(M['m01']/M['m00'])


#     max_contour = cv.approxPolyDP(max_contour,0.02*cv.arcLength(max_contour,True),True)
#     hull = cv.convexHull(max_contour)

#     for point in hull:
#         if cy > point[0][1]:
#             points1.append(tuple(point[0])) 

#     if debug:
#         cv.drawContours(img_result, [hull], 0, (0,255,0), 2)
#         for point in points1:
#             cv.circle(img_result, tuple(point), 15, [ 0, 0, 0], -1)


#   # STEP 6-2
#     hull = cv.convexHull(max_contour, returnPoints=False)
#     defects = cv.convexityDefects(max_contour, hull)

#     if defects is None:
#         return -1,None

#     points2=[]
#     for i in range(defects.shape[0]):
#         s,e,f,d = defects[i, 0]
#         start = tuple(max_contour[s][0])
#         end = tuple(max_contour[e][0])
#         far = tuple(max_contour[f][0])

#         angle = calculateAngle( np.array(start) - np.array(far), np.array(end) - np.array(far))

#     if angle < 90:
#         if start[1] < cy:
#             points2.append(start)
      
#         if end[1] < cy:
#             points2.append(end)

#     if debug:
#         cv.drawContours(img_result, [max_contour], 0, (255, 0, 255), 2)
#         for point in points2:
#             cv.circle(img_result, tuple(point), 20, [ 0, 255, 0], 5)


#   # STEP 6-3
#     points = points1 + points2
#     points = list(set(points))


#   # STEP 6-4
#     new_points = []
#     for p0 in points:

#         i = -1
#         for index,c0 in enumerate(max_contour):
#             c0 = tuple(c0[0])

#             if p0 == c0 or distanceBetweenTwoPoints(p0,c0)<20:
#                 i = index
#                 break

#         if i >= 0:
#             pre = i - 1
#             if pre < 0:
#                 pre = max_contour[len(max_contour)-1][0]
#             else:
#                 pre = max_contour[i-1][0]

#             next = i + 1
#             if next > len(max_contour)-1:
#                 next = max_contour[0][0]
#             else:
#                 next = max_contour[i+1][0]


#             if isinstance(pre, np.ndarray):
#                 pre = tuple(pre.tolist())
#             if isinstance(next, np.ndarray):
#                 next = tuple(next.tolist())

        
#             angle = calculateAngle( np.array(pre) - np.array(p0), np.array(next) - np.array(p0))     

#             if angle < 90:
#                 new_points.append(p0)
  
#     return 1,new_points


def process(img_bgr,high,low, debug):

    img_result = img_bgr.copy()

    # STEP 1
    img_bgr = removeFaceAra(img_bgr, cascade)


    # STEP 2
    img_binary = make_mask_image(img_bgr,high,low)


    # STEP 3
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7))
    img_binary = cv.morphologyEx(img_binary, cv.MORPH_CLOSE, kernel, 1)
    
#     # STEP 4
#     contours, hierarchy = cv.findContours(img_binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

#     if debug:
#         for cnt in contours:
#             cv.drawContours(img_result, [cnt], 0, (255, 0, 0), 3)  


#     # STEP 5
#     max_area, max_contour = findMaxArea(contours)  

#     if max_area == -1:
#         return img_result

#     if debug:
#         cv.drawContours(img_result, [max_contour], 0, (0, 0, 255), 3)  

    ret = 1
#     # STEP 6
#     ret,points = getFingerPosition(max_contour, img_result, debug)


#     # STEP 7
#     if ret > 0 and len(points) > 0:  
#         for point in points:
#             cv.circle(img_result, point, 20, [ 255, 0, 255], 5)

    return img_binary

# Point tracker

In [92]:
import time

from datetime import datetime



current_file_path = os.path.dirname(os.path.realpath("__file__"))
# cascade = cv.CascadeClassifier(cv.samples.findFile(current_file_path + "\haarcascade_frontalface_alt.xml"))

framecount = 1

# cap = cv.VideoCapture('test.avi')

cap = cv.VideoCapture(1)

high_b = (116, 255, 200)
low_b = (96, 100,  50)

high_y =(32, 255, 255) 
low_y= (12, 130, 150)

print("run")
start = time.time()

now = datetime.now()
its = str(now.date())+'_'+str(datetime.now().microsecond)

color_folder_name = 'color_'+its
yellow_folder_name = 'yellow_'+its
blue_folder_name='blue_'+its

os.makedirs(os.path.join(yellow_folder_name))
os.makedirs(os.path.join(blue_folder_name))
os.makedirs(os.path.join(color_folder_name))

print("make folder :")
print(color_folder_name)
print(yellow_folder_name)
print(blue_folder_name)


record=False


while True:
    
    ret,img_bgr = cap.read()
    
    cv.imshow("RGB",img_bgr)


    if ret == False:
        print('ret = False')
        break
        
    color_path = current_file_path + '\\'+color_folder_name+'\\'+ str(framecount).zfill(6)
    blue_label_path = current_file_path + '\\'+blue_folder_name+'\\'+ str(framecount).zfill(6)
    yellow_label_path = current_file_path + '\\'+yellow_folder_name+'\\'+ str(framecount).zfill(6)

    img_result_b = process(img_bgr,high_b,low_b, debug=False)
    img_result_y = process(img_bgr,high_y,low_y, debug=False)
    
        
    if record :
        np.save(color_path,img_bgr)
        np.save(blue_label_path,img_result_b)
        np.save(yellow_label_path,img_result_y)

    key = cv.waitKey(1)

    framecount= framecount +1
    
    if key == ord('r'):
        record = True      
        recorded= framecount
        print("start recording framd :" , framecount)

    if key== 27:
        break
        
    if key == ord('p'):
        cv2.waitKey(-1)
    
    cv.imshow("Result_b", img_result_b)
    cv.imshow("Result_y", img_result_y)

    
time_lengh=time.time() - start
cap.release()
cv.destroyAllWindows()
print()
print("Total Frame : ",framecount)
print("time :",time_lengh," sec")
print("FPS : ",framecount/time_lengh)
print()
if record:
    print("Record ",framecount -recorded,"frames")
else :
    print("there is no record, Delete folders")
    os.rmdir(os.path.join(color_folder_name))
    os.rmdir(os.path.join(blue_folder_name))
    os.rmdir(os.path.join(yellow_folder_name))
    
print("done")

run
make folder :
color_2020-09-01_187410
yellow_2020-09-01_187410
blue_2020-09-01_187410

Total Frame :  17660
time : 590.4169545173645  sec
FPS :  29.9110651631543

there is no record, Delete folders
done


In [91]:
import numpy as np
import os
import cv2
import matplotlib as plt

color_folder_path = current_file_path + '\\'+color_folder_name
blue_label_folder_path = current_file_path + '\\'+blue_folder_name
yellow_label_folder_path = current_file_path + '\\'+yellow_folder_name

color_list = os.listdir(color_folder_path) 
blue_label_list = os.listdir(blue_label_folder_path)
yellow_label_list = os.listdir(yellow_label_folder_path)

def maxpoint(label_fp):
    
    label = np.load(label_fp)
    
    row_sum = np.zeros(label.shape[1]).astype('float32')
    col_sum = np.zeros(label.shape[0]).astype('float32')
        
    for i,label_row in enumerate(label) :
        row_sum+=label_row
        
    for i in range(0,label.shape[1]):
        col_sum+=label[:,i]
    
    return row_sum.argmax(),col_sum.argmax()




for filename in yellow_label_list:
    
    color_fp = color_folder_path +'//' + filename
    blue_label_fp = blue_label_folder_path +'//' + filename
    yellow_label_fp = yellow_label_folder_path +'//' + filename
    
    
    
    color = np.load(color_fp)

    cv2.circle(color,maxpoint( blue_label_fp), 10, (0,255,0), -1)
    cv2.circle(color,maxpoint(yellow_label_fp), 10, (255,0,255), -1)
    
    cv2.imshow('point',color)
    #cv2.imshow('label',label)
    
    key = cv2.waitKey(1)
    
    if key ==27:
        break
    if key == ord('p'):
        cv2.waitKey(-1)

cv2.destroyAllWindows()